<a href="https://colab.research.google.com/github/AndrewDiv/PMfMIVV/blob/main/DiplomaUpd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import time
from scipy import optimize

# Simulator

In [ ]:
def Prevalence(host_num, single_strain_prob = 0.3):
  """ Simulate prevalence of strains in the host
    
  Parameters
  ----------
  host_num : int
    Number of patients
  single_strain_prob : float
    The probability that the host carries only one strain
       
  Returns
  -------
  prevs: ndarray, shape (host_num,)
    List of prevalence of one strain
  """
  prevs = np.zeros(host_num)
  for i in range(host_num):
    rnd = np.random.uniform()
    if rnd < single_strain_prob:
      prevs[i] = np.random.choice([0., 1.])
    else:
      prevs[i] = np.random.uniform()
  return prevs

In [ ]:
def pi_i(r_s, prevs, er_rate):
  return prevs*(r_s[0]*(1 - er_rate) + (1 - r_s[0])*er_rate) + (1-prevs)*(r_s[1]*(1 - er_rate) + (1 - r_s[1])*er_rate)

def SimulateReads(haps, prevs, coverage = 100,  er_rate = 0.01):
  """ Simulate reads from one host
    
  Parameters
  ----------
  haps : ndarray, shape (n, m)
    List of haplotypes
  prevs: ndarray, shape (k,)
    List of prevalences
  coverage: int
    Number of reads that cover sites
  er_rate: float
    Error rate 
       
  Returns
  -------
  data: ndarray, shape (host_num, hap_len, hap_num)
    Array of the number of observations of different alleles at each site
     for all patients
  """
  hap_len = len(haps[0])
  hap_num = len(haps)
  host_num = prevs.shape[0]
  data = np.empty((host_num, hap_len, hap_num))
  #print(hap_len, hap_num)
  for host in range(host_num):
    for snv in range(hap_len):
      als = np.zeros(hap_num)
      lcoverage = coverage
      als[1] = np.random.binomial(lcoverage, pi_i([haps[0][snv], haps[1][snv]], prevs[host], er_rate))
      als[0] = lcoverage - als[1]
      data[host][snv] = als
    
  return data

# Log-Likelihood

In [ ]:
def LogFactorial(n):
  return sum(np.log(np.arange(1, n+1)))

def Coeff(reads):
  coeff = 0
  for _reads in reads:
    coeff += LogFactorial(_reads[0] + _reads[1]) - LogFactorial(_reads[1]) - LogFactorial(_reads[0])
  return coeff

In [ ]:
def SingleHostLLH(prev, haps, samp_reads, er_rate=0.):
  """ Calculate minus log-likelihood of a single host
    
  Parameters
  ----------
  prev : float
    prevalence of 1st strain
  haps : ndarray, shape (hap_num, hap_len)
    List of haplotypes
  samp_reads: ndarray, shape (hap_len, hap_num)
    reads of host
  er_rate: float
    Error rate 
       
  Returns
  -------
  -llh: float
    log-likelihood of a single host
  """
  llh = Coeff(samp_reads)
  for snv in range(len(samp_reads)):
    p = prev*(haps[0][snv]*(1.0 - er_rate) + (1.0 - haps[0][snv])*er_rate) + (1-prev)*(haps[1][snv]*(1.0 - er_rate) + (1.0 - haps[1][snv])*er_rate)
    #if p <= 0:
    #  print("snv probs in haps: ", [haps[i][snv] for i in range(hap_num)])
    #  print("error rate: ", er_rate) 
    #  print("prevs:", prev)
    
    llh += samp_reads[snv][1] * np.log(p) + samp_reads[snv][0] * np.log(1.0 - p)
  return -llh

def LLH(haps, prevs, samples, num_of_haplotypes, er_rate):
  """ Calculate minus log-likelihood of the hosts
    
  Parameters
  ----------
  haps : ndarray, shape (hap_num, m)
    List of haplotypes
  prevs: ndarray, shape (k,)
    List of prevalences of 1st strain
  samples: ndarray, shape (n, m, hap_num)
    reads of all hosts
  num_of_haplotypes: int
    hap_num
  er_rate: float
    Error rate 
       
  Returns
  -------
  -llh: float
    log-likelihood of the whole sample
  """  
  haps = np.reshape(haps, (num_of_haplotypes, -1))
  #print(haps)
  llh = 0
  for sample_num in range(len(samples)):
      llh += SingleHostLLH(prevs[sample_num], haps, samples[sample_num], er_rate)
  #print('LLH:', llh)
  return llh

# Optimisation

In [ ]:
def optimise_prevs_const_haps(haps, prevs, samp_reads, hap_num=2, er_rate=0.):
  haps = np.reshape(haps, (hap_num, -1))
  #prevs = np.reshape(prevs, (len(samp_reads), -1))
  #print(prevs.shape)
  llh = 0.
  opt_prevs = np.empty(len(prevs))
  bnds = optimize.Bounds(0., 1.)
  #bnds = ((0, 1),)
  for host_num in range(len(samp_reads)):
    opt_res = optimize.minimize(SingleHostLLH, prevs[host_num], args=(haps, samp_reads[host_num], er_rate),
                                method='trust-constr', bounds=bnds, options = {'xtol': 1e-6, 'gtol': 1e-6, 'barrier_tol': 1e-6, 'maxiter': 100})
    opt_prevs[host_num] = opt_res.x
    llh += opt_res.fun
    
  return opt_prevs

In [ ]:
def optimise_haps_const_prevs(haps, prevs, samp_reads, num_of_hapl=2, er_rate=0.):
  #print(haps)
  #prevs = np.reshape(prevs, (len(samp_reads), -1))
  bnds = optimize.Bounds([0. for _ in range(len(haps))], [1. for _ in range(len(haps))])
  #bnds = ((0., 1.), ) * len(haps)
  opt_res = optimize.minimize(LLH, haps, args=(prevs, samp_reads, num_of_hapl, er_rate),
                                method='trust-constr', bounds=bnds, 
                                options = {'xtol': 1e-6, 'gtol': 1e-6, 'barrier_tol': 1e-6, 'maxiter': 100})
  opt_haps = opt_res.x

  return opt_haps

# Testing

In [ ]:
exmp_haplotypes = [[1., 1., 0., 0.],
              [0., 0., 1., 1.]] # Probabilistic description of haplotypes

hap_len = len(exmp_haplotypes[0]) # Number of sites
coverage = 100 # coverage sites
hap_num = len(exmp_haplotypes)  # Number of haplotypes(strains)
samp_num = 10  # Number of samples(patients)
er_rate = 0.00  # Error rate

np.random.seed(2022)

In [ ]:
exmp_prevs = Prevalence(samp_num, 0.3)
#print(exmp_prevs)

patients_data = SimulateReads(exmp_haplotypes, exmp_prevs, er_rate= 0.)
#print(patients_data)



In [ ]:
start_time = time.time()

haps_init = np.random.uniform(size = hap_num * hap_len)
prev_init = np.random.uniform(size = samp_num)


opt_haps = optimise_haps_const_prevs(haps_init, prev_init, patients_data)
opt_prevs = optimise_prevs_const_haps(opt_haps, prev_init, patients_data)

for _ in range(6):
    opt_haps = optimise_haps_const_prevs(opt_haps, opt_prevs, patients_data)
    opt_prevs = optimise_prevs_const_haps(opt_haps, opt_prevs, patients_data)

print("time elapsed: {:.2f}s".format(time.time() - start_time))

print('RESULT')
print("opt_haps:", np.reshape(opt_haps, (hap_num, -1)))
print("real_haps:", exmp_haplotypes)
print("opt_prevs:", opt_prevs)
print("real_prevs:", exmp_prevs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid

time elapsed: 16.85s
RESULT
opt_haps: [[0.97190829 0.98719267 0.01597062 0.01185343]
 [0.23274418 0.21420453 0.75323307 0.75671918]]
real_haps: [[1.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 1.0]]
opt_prevs: [9.99916430e-01 2.21104406e-02 5.11515332e-05 8.74051197e-01
 8.29995002e-01 8.17758885e-01 7.98641671e-01 2.02577751e-01
 1.17416054e-01 9.78389218e-01]
real_prevs: [1.         0.25531005 0.         0.89765723 0.89696312 0.83135342
 0.83357958 0.36804444 0.33950947 0.97752964]


#Simultaneous optimisation

In [ ]:
def SmlLLH(prev_haps, samp_reads, er_rate=0.):
  """ Calculate minus log-likelihood of a sample
    
  Parameters
  ----------
  prev_haps : ndarray, shape (n + hap_len*hap_num,)
    prevalence and haplotypes of sample
  samp_reads: ndarray, shape (n, hap_len, hap_num)
    reads of hosts
  er_rate: float
    Error rate 
       
  Returns
  -------
  -llh: float
    log-likelihood of sample
  """
  pn = samp_reads.shape[0]
  hl = samp_reads.shape[1]
  llh = 0.
  for sample_num in range(pn):
    llh += Coeff(samp_reads[sample_num])
    #print(Coeff(samp_reads[sample_num]))
    #print("llh:", llh)
    #print("llh:",len(llh))
    for snv in range(hl):
      p = prev_haps[sample_num]*(prev_haps[pn+snv]*(1.0 - er_rate) + (1.0 - prev_haps[pn+snv])*er_rate) + (1-prev_haps[sample_num])*(prev_haps[pn+hl+snv]*(1.0 - er_rate) + (1.0 - prev_haps[pn+hl+snv])*er_rate)
      #if p <= 0:
      #  print("snv probs in haps: ", [prev_haps[pn+snv], prev_haps[pn+hl+snv]])
      #  print("error rate: ", er_rate) 
      #  print("prevs:", prev_haps[sample_num])
      llh += samp_reads[sample_num][snv][1] * np.log(p) + samp_reads[sample_num][snv][0] * np.log(1.0 - p)
  return -llh

In [ ]:
def optimise_haps_and_prevs(prev_haps, samp_reads, er_rate=0.):
  bnds = optimize.Bounds([0. for _ in range(len(prev_haps))], [1. for _ in range(len(prev_haps))])
  #bnds = ((0., 1.), ) * len(prev_haps)
  #print(samp_reads.shape)
  opt_res = optimize.minimize(SmlLLH, prev_haps, args=(samp_reads, er_rate),
                                method='trust-constr', bounds=bnds, 
                                options = {'xtol': 1e-6, 'gtol': 1e-6, 'barrier_tol': 1e-6, 'maxiter': 100})
  opt_haps = opt_res.x

  return opt_haps

#Simultaneous optimisation Testing

In [ ]:
start_time = time.time()

#haps_init = np.random.uniform(size = hap_num * hap_len)
#prev_init = np.random.uniform(size = samp_num)
prev_haps_init = np.random.uniform(size = samp_num + hap_num*hap_len)

#opt_haps = optimise_haps_const_prevs(haps_init, prev_init, patients_data)
#opt_prevs = optimise_prevs_const_haps(opt_haps, prev_init, patients_data)
opt_prev_haps = optimise_haps_and_prevs(prev_haps_init, patients_data, er_rate)


for _ in range(6):
    opt_prev_haps = optimise_haps_and_prevs(opt_prev_haps, patients_data)

print("time elapsed: {:.2f}s".format(time.time() - start_time))

print('RESULT')
print("opt_prevs:", opt_prev_haps[:samp_num])
print("real_prevs:", exmp_prevs)
print("opt_haps:", np.reshape(opt_prev_haps[samp_num:], (hap_num, -1)))
print("real_haps:", exmp_haplotypes)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log


time elapsed: 13.98s
RESULT
opt_prevs: [9.99999986e-01 2.52499980e-01 1.53552038e-08 8.90000064e-01
 8.87500060e-01 8.45000097e-01 8.32500059e-01 3.85000005e-01
 3.22499992e-01 9.65000126e-01]
real_prevs: [1.         0.25531005 0.         0.89765723 0.89696312 0.83135342
 0.83357958 0.36804444 0.33950947 0.97752964]
opt_haps: [[9.99999701e-01 9.99999938e-01 4.73462121e-08 4.21744098e-08]
 [5.94266669e-08 4.40103940e-08 9.99999928e-01 9.99999925e-01]]
real_haps: [[1.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 1.0]]
